# Helmet Mapping 2d

In [1]:
# # Developer
# %cd /kaggle
# from IPython.display import clear_output
# pat = input('Personal Access Token:')
# clear_output()
# ! git clone https://$pat@github.com/qAp/nfl_helmet_assignment_kaggle.git

In [2]:
# %cd /kaggle/nfl_helmet_assignment_kaggle/
# ! git pull

In [3]:
# User
! cp -r /kaggle/input/nfl-helmet-assignment-kaggle/nfl_helmet_assignment_kaggle /kaggle/.

In [4]:
import os, sys
import time
import importlib
import random
from multiprocessing import Pool
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
from tqdm.auto import tqdm

sys.path.append('/kaggle/nfl_helmet_assignment_kaggle/')
from helmet_tracker.utils.features import add_track_features
from helmet_tracker.models.helmet_mapping import *

In [5]:
# import helmet_tracker
# importlib.reload(helmet_tracker.models.helmet_mapping)

# Basic data import

In [6]:
debug = True
BASE_DIR = '/kaggle/input/nfl-health-and-safety-helmet-assignment/'

labels = pd.read_csv(f'{BASE_DIR}/train_labels.csv')
if debug:
    dir_video = os.path.join(BASE_DIR, 'train')
    tracking = pd.read_csv(f'{BASE_DIR}/train_player_tracking.csv')
    helmets = pd.read_csv(f'{BASE_DIR}/train_baseline_helmets.csv')
    submission_df = pd.read_csv('/kaggle/input/nfl-mydata/submission_helmtrack_debug.csv')
else:
    dir_video = os.pth.join(BASE_DIR, 'test')
    tracking = pd.read_csv(f'{BASE_DIR}/test_player_tracking.csv')
    helmets = pd.read_csv(f'{BASE_DIR}/test_baseline_helmets.csv')
    submission_df = pd.read_csv('/kaggle/input/nfl-mydata/submission_helmtrack.csv')

In [7]:
%%time

# Add estimated video frame to NGS data

tracking = add_track_features(tracking) 

CPU times: user 1.09 s, sys: 84.8 ms, total: 1.18 s
Wall time: 1.2 s


In [8]:
if debug:
    sample_keys = submission_df['video_frame'].str.split('_').str[0].unique().astype(int)
    helmets['gameKey'] = helmets['video_frame'].str.split('_').str[0]
    tracking = tracking[tracking['gameKey'].isin(sample_keys)]
    helmets = helmets[helmets['gameKey'].astype(int).isin(sample_keys)]
    labels = labels[labels['gameKey'].astype(int).isin(sample_keys)]

# Map NGS player labels to helmets

In [9]:
CONF_THRE = 0.3

In [10]:
df_list = list(helmets.groupby('video_frame'))
df_list = [(video_frame, df, tracking, CONF_THRE) for video_frame, df in df_list]
len(df_list)

2596

In [11]:
%%time

def _mapping_df(args):
    video_frame, df, tracking, conf_thre = args
    return mapping_df_2d(video_frame, df, tracking, conf_thre=conf_thre)

# df_tgt_2d = _mapping_df(df_list[2019])

p = Pool(processes=os.cpu_count())
submission_df_list = []
with tqdm(total=len(df_list)) as pbar:
    for this_df in p.imap(_mapping_df, df_list):
        submission_df_list.append(this_df)
        pbar.update(1)
p.close()

  0%|          | 0/2596 [00:00<?, ?it/s]

CPU times: user 55.7 s, sys: 5.69 s, total: 1min 1s
Wall time: 46min 14s


In [12]:
submission_df = pd.concat(submission_df_list)

p = '/kaggle/working/' + ('submission_helmtrack2d_debug.csv' if debug else
                          'submission_helmtrack2d.csv')
submission_df.to_csv(p, index=False)